Make sure to have at least Tesla T4 GPU for faster finetuning. Google colab pro offers T4 GPU.

In [10]:
!nvidia-smi

Sun Feb 19 16:52:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    31W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip3 install transformers==4.26.1
!git clone --depth 1 --branch v4.26.1 https://github.com/huggingface/transformers
!pip install sentencepiece
!pip install datasets
!pip3 install evaluate
!pip install farasapy
!pip install pyarabic
!pip install fuzzysearch
!pip3 install camel_tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 80.8 MB/s eta 0:00:00
Cloning into 'transformers'...
remote: Enumerating objects: 3271, done.
remote: Counting objects: 100% (3271/3271), done.
remote: Compressing objects: 100% (2711/2711), done.
remote: Total 3271 (delta 1008), reused 1124 (delta 511), pack-reused 0
Receiving objects: 100% (3271/3271), 11.37 MiB | 10.86 MiB/s, done.
Resolving deltas: 100% (1008/1008), done.
Note: switching to 'ae54e3c3b18bac0832ad62ea9b896dfd52a09850'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a n

# **ArabicTyDi QA**

First lets process Arabic TyDi QA dataset. TyDi combine all questions for more than 11 different language so we need to extract the arabic portion and also do some pre-processing. All credits to AUB Mind LAB.

In [2]:
!git clone https://github.com/aub-mind/arabert
!cp arabert/examples/question-answering/utils_qa.py .
!cp arabert/examples/question-answering/trainer_qa.py .
!cp arabert/examples/question-answering/run_qa.py .
!cp arabert/examples/question-answering/squad_preprocessing.py .

Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 18.76 MiB/s, done.
Resolving deltas: 100% (339/339), done.


In [3]:
!wget https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-train.json
!wget https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-dev.json

--2023-02-19 16:07:39--  https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-train.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58004076 (55M) [application/json]
Saving to: ‘tydiqa-goldp-v1.1-train.json’

tydiqa-goldp-v1.1-t 100%[===================>]  55.32M  22.3MB/s    in 2.5s    

2023-02-19 16:07:42 (22.3 MB/s) - ‘tydiqa-goldp-v1.1-train.json’ saved [58004076/58004076]

--2023-02-19 16:07:42--  https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-dev.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 64.233.189.128, 108.177.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5617409 (5.4M) [application/js

In [4]:
model_name="sultan/ArabicT5-Large"

In [5]:
!rm -rf *-pre.json
!python squad_preprocessing.py \
  --input_file "tydiqa-goldp-v1.1-train.json" \
  --output_file "tydiqa-goldp-v1.1-train-pre.json" \
  --model_name=$model_name \
  --filter_tydiqa=True

2023-02-19 16:07:44.064205: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 16:07:45.038861: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 16:07:45.038946: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 16:07:45.038961: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [6]:
!python squad_preprocessing.py \
  --input_file "tydiqa-goldp-v1.1-dev.json" \
  --output_file "tydiqa-goldp-v1.1-dev-pre.json" \
  --model_name=$model_name \
  --filter_tydiqa=True

2023-02-19 16:08:14.518312: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 16:08:15.495693: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 16:08:15.495797: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 16:08:15.495809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

click on the play button on the code below so the code will saved to this colab as "artydiqa.py"

In [7]:
#@title
%%writefile artydiqa.py
"""TODO(tydiqa): Add a description here."""

from __future__ import absolute_import, division, print_function

import json
import textwrap

import datasets


# TODO(tydiqa): BibTeX citation
_CITATION = """\
@article{tydiqa,
title   = {TyDi QA: A Benchmark for Information-Seeking Question Answering in Typologically Diverse Languages},
author  = {Jonathan H. Clark and Eunsol Choi and Michael Collins and Dan Garrette and Tom Kwiatkowski and Vitaly Nikolaev and Jennimaria Palomaki}
year    = {2020},
journal = {Transactions of the Association for Computational Linguistics}
}
"""

# TODO(tydiqa):
_DESCRIPTION = """\
TyDi QA is a question answering dataset covering 11 typologically diverse languages with 204K question-answer pairs.
The languages of TyDi QA are diverse with regard to their typology -- the set of linguistic features that each language
expresses -- such that we expect models performing well on this set to generalize across a large number of the languages
in the world. It contains language phenomena that would not be found in English-only corpora. To provide a realistic
information-seeking task and avoid priming effects, questions are written by people who want to know the answer, but
don’t know the answer yet, (unlike SQuAD and its descendents) and the data is collected directly in each language without
the use of translation (unlike MLQA and XQuAD).
"""

# _URL = "https://storage.googleapis.com/tydiqa/"

# _PRIMARY_URLS = {
#     "train": _URL + "v1.0/tydiqa-v1.0-train.jsonl.gz",
#     "dev": _URL + "v1.0/tydiqa-v1.0-dev.jsonl.gz",
# }
# _SECONDARY_URLS = {
#     "train": _URL + "v1.1/tydiqa-goldp-v1.1-train.json",
#     "dev": _URL + "v1.1/tydiqa-goldp-v1.1-dev.json",
# }

#use this for AraBERTv1 and V2
_URL = "https://storage.googleapis.com/tydiqa/"
_URL2 = "/content/"
_PRIMARY_URLS = {
    "train": _URL + "v1.0/tydiqa-v1.0-train.jsonl.gz",
    "dev": _URL + "v1.0/tydiqa-v1.0-dev.jsonl.gz",
}
_SECONDARY_URLS = {
    "train": _URL2 + "tydiqa-goldp-v1.1-train-pre.json",
    "dev": _URL2 + "tydiqa-goldp-v1.1-dev-pre.json",
}


class TydiqaConfig(datasets.BuilderConfig):

    """ BuilderConfig for Tydiqa"""

    def __init__(self, **kwargs):
        """

        Args:
            **kwargs: keyword arguments forwarded to super.
        """
        super(TydiqaConfig, self).__init__(version=datasets.Version("1.0.0", ""), **kwargs)


class Tydiqa(datasets.GeneratorBasedBuilder):
    """TODO(tydiqa): Short description of my dataset."""

    # TODO(tydiqa): Set up version.
    VERSION = datasets.Version("0.1.0")
    BUILDER_CONFIGS = [
        TydiqaConfig(
            name="primary_task",
            description=textwrap.dedent(
                """\
          Passage selection task (SelectP): Given a list of the passages in the article, return either (a) the index of
          the passage that answers the question or (b) NULL if no such passage exists.
          Minimal answer span task (MinSpan): Given the full text of an article, return one of (a) the start and end
          byte indices of the minimal span that completely answers the question; (b) YES or NO if the question requires
          a yes/no answer and we can draw a conclusion from the passage; (c) NULL if it is not possible to produce a
          minimal answer for this question."""
            ),
        ),
        TydiqaConfig(
            name="secondary_task",
            description=textwrap.dedent(
                """Gold passage task (GoldP): Given a passage that is guaranteed to contain the
          answer, predict the single contiguous span of characters that answers the question. This is more similar to
          existing reading comprehension datasets (as opposed to the information-seeking task outlined above).
          This task is constructed with two goals in mind: (1) more directly comparing with prior work and (2) providing
          a simplified way for researchers to use TyDi QA by providing compatibility with existing code for SQuAD 1.1,
          XQuAD, and MLQA. Toward these goals, the gold passage task differs from the primary task in several ways:
          only the gold answer passage is provided rather than the entire Wikipedia article;
          unanswerable questions have been discarded, similar to MLQA and XQuAD;
          we evaluate with the SQuAD 1.1 metrics like XQuAD; and
         Thai and Japanese are removed since the lack of whitespace breaks some tools.
          """
            ),
        ),
    ]

    def _info(self):
        # TODO(tydiqa): Specifies the datasets.DatasetInfo object
        if self.config.name == "primary_task":
            return datasets.DatasetInfo(
                # This is the description that will appear on the datasets page.
                description=_DESCRIPTION,
                # datasets.features.FeatureConnectors
                features=datasets.Features(
                    {
                        "passage_answer_candidates": datasets.features.Sequence(
                            {
                                "plaintext_start_byte": datasets.Value("int32"),
                                "plaintext_end_byte": datasets.Value("int32"),
                            }
                        ),
                        "question_text": datasets.Value("string"),
                        "document_title": datasets.Value("string"),
                        "language": datasets.Value("string"),
                        "annotations": datasets.features.Sequence(
                            {
                                # 'annotation_id': datasets.Value('variant'),
                                "passage_answer_candidate_index": datasets.Value("int32"),
                                "minimal_answers_start_byte": datasets.Value("int32"),
                                "minimal_answers_end_byte": datasets.Value("int32"),
                                "yes_no_answer": datasets.Value("string"),
                            }
                        ),
                        "document_plaintext": datasets.Value("string"),
                        # 'example_id': datasets.Value('variant'),
                        "document_url": datasets.Value("string")
                        # These are the features of your dataset like images, labels ...
                    }
                ),
                # If there's a common (input, target) tuple from the features,
                # specify them here. They'll be used if as_supervised=True in
                # builder.as_dataset.
                supervised_keys=None,
                # Homepage of the dataset for documentation
                homepage="https://github.com/google-research-datasets/tydiqa",
                citation=_CITATION,
            )
        elif self.config.name == "secondary_task":
            return datasets.DatasetInfo(
                description=_DESCRIPTION,
                features=datasets.Features(
                    {
                        "id": datasets.Value("string"),
                        "title": datasets.Value("string"),
                        "context": datasets.Value("string"),
                        "question": datasets.Value("string"),
                        "answers": datasets.features.Sequence(
                            {
                                "text": datasets.Value("string"),
                                "answer_start": datasets.Value("int32"),
                            }
                        ),
                    }
                ),
                # No default supervised_keys (as we have to pass both question
                # and context as input).
                supervised_keys=None,
                homepage="https://github.com/google-research-datasets/tydiqa",
                citation=_CITATION,
            )

    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        # TODO(tydiqa): Downloads the data and defines the splits
        # dl_manager is a datasets.download.DownloadManager that can be used to
        # download and extract URLs
        primary_downloaded = dl_manager.download_and_extract(_PRIMARY_URLS)
        secondary_downloaded = dl_manager.download_and_extract(_SECONDARY_URLS)
        if self.config.name == "primary_task":
            return [
                datasets.SplitGenerator(
                    name=datasets.Split.TRAIN,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": primary_downloaded["train"]},
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.VALIDATION,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": primary_downloaded["dev"]},
                ),
            ]
        elif self.config.name == "secondary_task":
            return [
                datasets.SplitGenerator(
                    name=datasets.Split.TRAIN,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": secondary_downloaded["train"]},
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.VALIDATION,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": secondary_downloaded["dev"]},
                ),
            ]

    def _generate_examples(self, filepath):
        """Yields examples."""
        # TODO(tydiqa): Yields (key, example) tuples from the dataset
        if self.config.name == "primary_task":
            with open(filepath, encoding="utf-8") as f:
                for id_, row in enumerate(f):
                    data = json.loads(row)
                    passages = data["passage_answer_candidates"]
                    end_byte = [passage["plaintext_end_byte"] for passage in passages]
                    start_byte = [passage["plaintext_start_byte"] for passage in passages]
                    title = data["document_title"]
                    lang = data["language"]
                    question = data["question_text"]
                    annotations = data["annotations"]
                    # annot_ids = [annotation["annotation_id"] for annotation in annotations]
                    yes_no_answers = [annotation["yes_no_answer"] for annotation in annotations]
                    min_answers_end_byte = [
                        annotation["minimal_answer"]["plaintext_end_byte"] for annotation in annotations
                    ]
                    min_answers_start_byte = [
                        annotation["minimal_answer"]["plaintext_start_byte"] for annotation in annotations
                    ]
                    passage_cand_answers = [
                        annotation["passage_answer"]["candidate_index"] for annotation in annotations
                    ]
                    doc = data["document_plaintext"]
                    # example_id = data["example_id"]
                    url = data["document_url"]
                    yield id_, {
                        "passage_answer_candidates": {
                            "plaintext_start_byte": start_byte,
                            "plaintext_end_byte": end_byte,
                        },
                        "question_text": question,
                        "document_title": title,
                        "language": lang,
                        "annotations": {
                            # 'annotation_id': annot_ids,
                            "passage_answer_candidate_index": passage_cand_answers,
                            "minimal_answers_start_byte": min_answers_start_byte,
                            "minimal_answers_end_byte": min_answers_end_byte,
                            "yes_no_answer": yes_no_answers,
                        },
                        "document_plaintext": doc,
                        # 'example_id': example_id,
                        "document_url": url,
                    }
        elif self.config.name == "secondary_task":
            with open(filepath, encoding="utf-8") as f:
                data = json.load(f)
                for article in data["data"]:
                    title = article.get("title", "").strip()
                    for paragraph in article["paragraphs"]:
                        context = paragraph["context"].strip()
                        for qa in paragraph["qas"]:
                            question = qa["question"].strip()
                            id_ = qa["id"]
                            if "arabic" not in id_:
                              continue
                            answer_starts = [answer["answer_start"] for answer in qa["answers"] if answer["answer_start"] != -1]                            
                            answers = [answer["text"].strip() for answer in qa["answers"] if answer["answer_start"] != -1]
                            if len(answers) == 0 or len(answer_starts)==0:
                              print("question skipped")
                              continue
                            assert len(answer_starts)==len(answers)
                            # Features currently used are "context", "question", and "answers".
                            # Others are extracted here for the ease of future expansions.
                            yield id_, {
                                "title": title,
                                "context": context,
                                "question": question,
                                "id": id_,
                                "answers": {
                                    "answer_start": answer_starts,
                                    "text": answers,
                                },
                            }

Writing artydiqa.py


In [18]:
!python3 /content/transformers/examples/pytorch/question-answering/run_qa.py \
	--output_dir out \
	--model_name_or_path sultan/ArabicTransformer-intermediate \
	--dataset_name artydiqa.py \
	--dataset_config_name "secondary_task" \
	--do_train --do_eval \
	--per_device_train_batch_size 16 \
  --learning_rate 3e-5 \
  --num_train_epochs 4 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir out \
  --n_best_size 20 \
  --evaluation_strategy epoch \
  --save_steps 500000 \
  --overwrite_output_dir \
  --seed 42 \
  --warmup_steps 500 \
	--fp16

2023-02-19 17:02:55.047535: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 17:02:55.938577: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 17:02:55.938696: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 17:02:55.938717: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

As you can see from the evaluation results, we achieved 87.22/75.57 F1/EM scores which slightly exceeded our reported results in the paper.

# **Quran_Qa dataset**

https://sites.google.com/view/quran-qa-2022

In [27]:
!wget -O qrcd_v1.1_dev.jsonl https://gitlab.com/bigirqu/quranqa/-/raw/main/datasets/qrcd_v1.1_dev.jsonl?inline=false
!wget -O qrcd_v1.1_train.jsonl https://gitlab.com/bigirqu/quranqa/-/raw/main/datasets/qrcd_v1.1_train.jsonl?inline=false
!wget -O qrcd_v1.1_test.jsonl https://gitlab.com/bigirqu/quranqa/-/raw/main/datasets/qrcd_v1.1_test_gold.jsonl?inline=false

--2023-02-19 18:10:55--  https://gitlab.com/bigirqu/quranqa/-/raw/main/datasets/qrcd_v1.1_dev.jsonl?inline=false
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117412 (115K) [application/octet-stream]
Saving to: ‘qrcd_v1.1_dev.jsonl’

qrcd_v1.1_dev.jsonl 100%[===================>] 114.66K  --.-KB/s    in 0.02s   

2023-02-19 18:10:56 (4.64 MB/s) - ‘qrcd_v1.1_dev.jsonl’ saved [117412/117412]

--2023-02-19 18:10:56--  https://gitlab.com/bigirqu/quranqa/-/raw/main/datasets/qrcd_v1.1_train.jsonl?inline=false
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 756171 (738K) [application/octet-stream]
Saving to: ‘qrcd_v1.1_train.jsonl’

qrcd_v1.1_train.jso 100%[=========

In [57]:
import json
import pandas as pd 
def quran_jsonl_to_json(quran_data,squad_style_file):
  quaran_pd_data=pd.read_json(quran_data,lines=True)
  quran_qa_json_entry=[]
  for index,row in quaran_pd_data.iterrows():
      quran_qa_json_entry.append({"title":"surah","paragraphs":[{"qas":[{"question":row["question"], "id":row["pq_id"],"answers":row["answers"]}],"context":row["passage"]}]})
  with open(squad_style_file, 'w') as fp:
      json.dump({"data":quran_qa_json_entry,"version":"Quran_Dataset_SQuAD_Style1.1"}, fp)
quran_jsonl_to_json("qrcd_v1.1_train.jsonl","qrcd_v1.1_train.json")
quran_jsonl_to_json("qrcd_v1.1_dev.jsonl","qrcd_v1.1_dev.json")
quran_jsonl_to_json("qrcd_v1.1_test.jsonl","qrcd_v1.1_test.json")
## we need to replace "start_char" key in quran_qa dataset with "answer_start" key to match tydiqa and squad style.
!sed -i 's/start_char/answer_start/g' qrcd_v1.1_train.json
!sed -i 's/start_char/answer_start/g' qrcd_v1.1_dev.json
!sed -i 's/start_char/answer_start/g' qrcd_v1.1_test.json

click on the play button on the code below so the code will saved to this colab as "quran_qa.py"

In [58]:
#@title
%%writefile quran_qa.py
"""TODO(tydiqa): Add a description here."""

from __future__ import absolute_import, division, print_function

import json
import textwrap

import datasets


# TODO(tydiqa): BibTeX citation
_CITATION = """\
@article{tydiqa,
title   = {TyDi QA: A Benchmark for Information-Seeking Question Answering in Typologically Diverse Languages},
author  = {Jonathan H. Clark and Eunsol Choi and Michael Collins and Dan Garrette and Tom Kwiatkowski and Vitaly Nikolaev and Jennimaria Palomaki}
year    = {2020},
journal = {Transactions of the Association for Computational Linguistics}
}
"""

# TODO(tydiqa):
_DESCRIPTION = """\
TyDi QA is a question answering dataset covering 11 typologically diverse languages with 204K question-answer pairs.
The languages of TyDi QA are diverse with regard to their typology -- the set of linguistic features that each language
expresses -- such that we expect models performing well on this set to generalize across a large number of the languages
in the world. It contains language phenomena that would not be found in English-only corpora. To provide a realistic
information-seeking task and avoid priming effects, questions are written by people who want to know the answer, but
don’t know the answer yet, (unlike SQuAD and its descendents) and the data is collected directly in each language without
the use of translation (unlike MLQA and XQuAD).
"""

# _URL = "https://storage.googleapis.com/tydiqa/"

# _PRIMARY_URLS = {
#     "train": _URL + "v1.0/tydiqa-v1.0-train.jsonl.gz",
#     "dev": _URL + "v1.0/tydiqa-v1.0-dev.jsonl.gz",
# }
# _SECONDARY_URLS = {
#     "train": _URL + "v1.1/tydiqa-goldp-v1.1-train.json",
#     "dev": _URL + "v1.1/tydiqa-goldp-v1.1-dev.json",
# }

#use this for AraBERTv1 and V2
_URL = "https://storage.googleapis.com/tydiqa/"
_URL2 = "/content/"
_PRIMARY_URLS = {
    "train": _URL + "v1.0/tydiqa-v1.0-train.jsonl.gz",
    "dev": _URL + "v1.0/tydiqa-v1.0-dev.jsonl.gz",
}
_SECONDARY_URLS = {
    "train": _URL2 + "qrcd_v1.1_train.json",
    "dev": _URL2 + "qrcd_v1.1_dev.json",
    "test": _URL2 + "qrcd_v1.1_test.json",
}


class TydiqaConfig(datasets.BuilderConfig):

    """ BuilderConfig for Tydiqa"""

    def __init__(self, **kwargs):
        """

        Args:
            **kwargs: keyword arguments forwarded to super.
        """
        super(TydiqaConfig, self).__init__(version=datasets.Version("1.0.0", ""), **kwargs)


class Tydiqa(datasets.GeneratorBasedBuilder):
    """TODO(tydiqa): Short description of my dataset."""

    # TODO(tydiqa): Set up version.
    VERSION = datasets.Version("0.1.0")
    BUILDER_CONFIGS = [
        TydiqaConfig(
            name="primary_task",
            description=textwrap.dedent(
                """\
          Passage selection task (SelectP): Given a list of the passages in the article, return either (a) the index of
          the passage that answers the question or (b) NULL if no such passage exists.
          Minimal answer span task (MinSpan): Given the full text of an article, return one of (a) the start and end
          byte indices of the minimal span that completely answers the question; (b) YES or NO if the question requires
          a yes/no answer and we can draw a conclusion from the passage; (c) NULL if it is not possible to produce a
          minimal answer for this question."""
            ),
        ),
        TydiqaConfig(
            name="secondary_task",
            description=textwrap.dedent(
                """Gold passage task (GoldP): Given a passage that is guaranteed to contain the
          answer, predict the single contiguous span of characters that answers the question. This is more similar to
          existing reading comprehension datasets (as opposed to the information-seeking task outlined above).
          This task is constructed with two goals in mind: (1) more directly comparing with prior work and (2) providing
          a simplified way for researchers to use TyDi QA by providing compatibility with existing code for SQuAD 1.1,
          XQuAD, and MLQA. Toward these goals, the gold passage task differs from the primary task in several ways:
          only the gold answer passage is provided rather than the entire Wikipedia article;
          unanswerable questions have been discarded, similar to MLQA and XQuAD;
          we evaluate with the SQuAD 1.1 metrics like XQuAD; and
         Thai and Japanese are removed since the lack of whitespace breaks some tools.
          """
            ),
        ),
    ]

    def _info(self):
        # TODO(tydiqa): Specifies the datasets.DatasetInfo object
        if self.config.name == "primary_task":
            return datasets.DatasetInfo(
                # This is the description that will appear on the datasets page.
                description=_DESCRIPTION,
                # datasets.features.FeatureConnectors
                features=datasets.Features(
                    {
                        "passage_answer_candidates": datasets.features.Sequence(
                            {
                                "plaintext_start_byte": datasets.Value("int32"),
                                "plaintext_end_byte": datasets.Value("int32"),
                            }
                        ),
                        "question_text": datasets.Value("string"),
                        "document_title": datasets.Value("string"),
                        "language": datasets.Value("string"),
                        "annotations": datasets.features.Sequence(
                            {
                                # 'annotation_id': datasets.Value('variant'),
                                "passage_answer_candidate_index": datasets.Value("int32"),
                                "minimal_answers_start_byte": datasets.Value("int32"),
                                "minimal_answers_end_byte": datasets.Value("int32"),
                                "yes_no_answer": datasets.Value("string"),
                            }
                        ),
                        "document_plaintext": datasets.Value("string"),
                        # 'example_id': datasets.Value('variant'),
                        "document_url": datasets.Value("string")
                        # These are the features of your dataset like images, labels ...
                    }
                ),
                # If there's a common (input, target) tuple from the features,
                # specify them here. They'll be used if as_supervised=True in
                # builder.as_dataset.
                supervised_keys=None,
                # Homepage of the dataset for documentation
                homepage="https://github.com/google-research-datasets/tydiqa",
                citation=_CITATION,
            )
        elif self.config.name == "secondary_task":
            return datasets.DatasetInfo(
                description=_DESCRIPTION,
                features=datasets.Features(
                    {
                        "id": datasets.Value("string"),
                        "title": datasets.Value("string"),
                        "context": datasets.Value("string"),
                        "question": datasets.Value("string"),
                        "answers": datasets.features.Sequence(
                            {
                                "text": datasets.Value("string"),
                                "answer_start": datasets.Value("int32"),
                            }
                        ),
                    }
                ),
                # No default supervised_keys (as we have to pass both question
                # and context as input).
                supervised_keys=None,
                homepage="https://github.com/google-research-datasets/tydiqa",
                citation=_CITATION,
            )

    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        # TODO(tydiqa): Downloads the data and defines the splits
        # dl_manager is a datasets.download.DownloadManager that can be used to
        # download and extract URLs
        primary_downloaded = dl_manager.download_and_extract(_PRIMARY_URLS)
        secondary_downloaded = dl_manager.download_and_extract(_SECONDARY_URLS)
        if self.config.name == "primary_task":
            return [
                datasets.SplitGenerator(
                    name=datasets.Split.TRAIN,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": primary_downloaded["train"]},
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.VALIDATION,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": primary_downloaded["dev"]},
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.TEST,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": primary_downloaded["test"]},
                ),
            ]
        elif self.config.name == "secondary_task":
            return [
                datasets.SplitGenerator(
                    name=datasets.Split.TRAIN,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": secondary_downloaded["train"]},
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.VALIDATION,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": secondary_downloaded["dev"]},
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.TEST,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": secondary_downloaded["test"]},
                ),
            ]

    def _generate_examples(self, filepath):
        """Yields examples."""
        # TODO(tydiqa): Yields (key, example) tuples from the dataset
        if self.config.name == "primary_task":
            with open(filepath, encoding="utf-8") as f:
                for id_, row in enumerate(f):
                    data = json.loads(row)
                    passages = data["passage_answer_candidates"]
                    end_byte = [passage["plaintext_end_byte"] for passage in passages]
                    start_byte = [passage["plaintext_start_byte"] for passage in passages]
                    title = data["document_title"]
                    lang = data["language"]
                    question = data["question_text"]
                    annotations = data["annotations"]
                    # annot_ids = [annotation["annotation_id"] for annotation in annotations]
                    yes_no_answers = [annotation["yes_no_answer"] for annotation in annotations]
                    min_answers_end_byte = [
                        annotation["minimal_answer"]["plaintext_end_byte"] for annotation in annotations
                    ]
                    min_answers_start_byte = [
                        annotation["minimal_answer"]["plaintext_start_byte"] for annotation in annotations
                    ]
                    passage_cand_answers = [
                        annotation["passage_answer"]["candidate_index"] for annotation in annotations
                    ]
                    doc = data["document_plaintext"]
                    # example_id = data["example_id"]
                    url = data["document_url"]
                    yield id_, {
                        "passage_answer_candidates": {
                            "plaintext_start_byte": start_byte,
                            "plaintext_end_byte": end_byte,
                        },
                        "question_text": question,
                        "document_title": title,
                        "language": lang,
                        "annotations": {
                            # 'annotation_id': annot_ids,
                            "passage_answer_candidate_index": passage_cand_answers,
                            "minimal_answers_start_byte": min_answers_start_byte,
                            "minimal_answers_end_byte": min_answers_end_byte,
                            "yes_no_answer": yes_no_answers,
                        },
                        "document_plaintext": doc,
                        # 'example_id': example_id,
                        "document_url": url,
                    }
        elif self.config.name == "secondary_task":
            with open(filepath, encoding="utf-8") as f:
                data = json.load(f)
                for article in data["data"]:
                    title = article.get("title", "").strip()
                    for paragraph in article["paragraphs"]:
                        context = paragraph["context"].strip()
                        for qa in paragraph["qas"]:
                            question = qa["question"].strip()
                            id_ = qa["id"]
                            answer_starts = [answer["answer_start"] for answer in qa["answers"] if answer["answer_start"] != -1]                            
                            answers = [answer["text"].strip() for answer in qa["answers"] if answer["answer_start"] != -1]
                            if len(answers) == 0 or len(answer_starts)==0:
                              print("question skipped")
                              continue
                            assert len(answer_starts)==len(answers)
                            # Features currently used are "context", "question", and "answers".
                            # Others are extracted here for the ease of future expansions.
                            yield id_, {
                                "title": title,
                                "context": context,
                                "question": question,
                                "id": id_,
                                "answers": {
                                    "answer_start": answer_starts,
                                    "text": answers,
                                },
                            }

Overwriting quran_qa.py


Since quran_qa dataset is small, you will use the checkpoint that we finetuned on ArabicTyDi QA dataset from previous step (we saved it to [out] directory)

In [60]:
!python3 /content/transformers/examples/pytorch/question-answering/run_qa.py \
	--output_dir quran_qa_model \
	--model_name_or_path out \
	--dataset_name quran_qa.py \
	--dataset_config_name "secondary_task" \
	--do_train --do_eval --do_predict \
	--per_device_train_batch_size 16 \
  --learning_rate 3e-5 \
  --num_train_epochs 10 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir out \
  --n_best_size 20 \
  --evaluation_strategy epoch \
  --save_steps 500000 \
  --overwrite_output_dir \
  --overwrite_cache \
  --seed 42 \
  --warmup_steps 500 \
	--fp16 

2023-02-19 18:36:34.644681: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 18:36:35.584964: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 18:36:35.585071: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-19 18:36:35.585088: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War